Versión final CA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import random
import matplotlib.patches as mpatches
import pandas as pd
from IPython.display import display as ipy_display
import matplotlib as mpl

seed = 25   #10
np.random.seed(seed)
# ------------------------------
# Constants
# ------------------------------
EMPTY = 0
BIRD = 1
FOOD = 2
PROHIBITED = 3
DESTINATION = 4

# ------------------------------
# Settings
# ------------------------------
grid_size = (250, 250)
p_food = 0.5
p_prohibited = 0.1

start = (0, 0)    
destination = (249, 249)   
wind_change_interval = 30

# ------------------------------
# Environment Setup
# ------------------------------
"""Aquí se definen los estados del entorno:
- EMPTY = 0
- BIRD = 1
- FOOD = 2
- PROHIBITED = 3
- DESTINATION = 4
El entorno es una cuadrícula de tamaño grid_size, donde se colocan el pájaro, 
la comida, las áreas prohibidas y el destino.
El pájaro comienza en la posición start y debe llegar a destination, 
recolectando comida y evitando áreas prohibidas.
El viento se simula con una cuadrícula de enteros aleatorios entre -5 y 5, 
que afecta las decisiones del pájaro.
"""

state_grid = np.zeros(grid_size, dtype=int)
state_grid[start] = BIRD
state_grid[destination] = DESTINATION

food_cells = np.random.rand(*grid_size) < p_food
state_grid[food_cells] = FOOD

prohibited_cells = (np.random.rand(*grid_size) < p_prohibited) & (state_grid == EMPTY)
state_grid[prohibited_cells] = PROHIBITED

wind_grid = np.random.randint(-5, 6, size=grid_size)

trajectory = [start]
move_history = []
food_collected = 0

# ------------------------------
# Helper Functions
# ------------------------------
"""Estas funciones ayudan a obtener los vecinos de una celda, detectar oscilaciones en el movimiento del pájaro,
proponer movimientos basados en el estado del entorno y resolver los movimientos propuestos actualizando el estado del entorno.
get_neighbors(pos) devuelve una lista de celdas vecinas a la posición dada.
detect_oscillation(history) verifica si hay un patrón de oscilación en el historial de movimientos del pájaro.
propose_moves(state_grid, wind_grid, oscillating) genera propuestas de movimiento para el pájaro basadas en el estado del entorno,
la cuadrícula de viento y si hay oscilaciones detectadas.
resolve_and_update(state_grid, proposals) aplica las propuestas de movimiento al estado del entorno,
actualizando la posición del pájaro y recolectando comida si es necesario.
"""

def get_neighbors(pos):
    x, y = pos
    return [(i, j) for i, j in [(x+1, y), (x-1, y), (x, y+1), (x, y-1)] if 0 <= i < grid_size[0] and 0 <= j < grid_size[1]]

def detect_oscillation(history):
    if len(history) < 4:
        return False
    pattern = history[-4:]
    count_ns = sum(
        1 for i in range(0, len(pattern) - 1, 2)
        if pattern[i] == (-pattern[i+1][0], -pattern[i+1][1]) and (pattern[i][0] == 0 or pattern[i][1] == 0)
    )
    return count_ns >= 3

def propose_moves(state_grid, wind_grid, oscillating):
    """
    Movimiento de ave con sesgo de brújula interna
    Proponer movimientos para las aves basados en el estado del entorno, la cuadrícula de viento y si hay oscilaciones detectadas.
    Se evalúan las celdas vecinas y se asigna una puntuación a cada movimiento basado en la comida, el viento, la distancia al destino y el sesgo de brújula.
    """

    proposals = {}
    bird_positions = np.argwhere(state_grid == BIRD)
    for x, y in bird_positions:
        neighbors = get_neighbors((x, y))
        best_score = -np.inf
        best_moves = []
        for nx, ny in neighbors:
            if state_grid[nx, ny] in [EMPTY, FOOD, DESTINATION]:
                dist_penalty = (abs(destination[0] - nx) + abs(destination[1] - ny)) * 0.5
                food_bonus = 10 if state_grid[nx, ny] == FOOD else 0
                wind_score = wind_grid[nx, ny]
                
                # Internal Compass Bias
                dx_dest = destination[0] - x
                dy_dest = destination[1] - y
                dx_move = nx - x
                dy_move = ny - y
                norm = max(abs(dx_dest), abs(dy_dest), 1)
                dir_alignment = (dx_dest * dx_move + dy_dest * dy_move) / norm
                compass_bias = 5 * dir_alignment

                osc_penalty = -5 if oscillating and abs(nx - x) > 0 else 0

                score = food_bonus + wind_score - dist_penalty + compass_bias + osc_penalty

                if score > best_score:
                    best_score = score
                    best_moves = [(nx, ny)]
                elif score == best_score:
                    best_moves.append((nx, ny))
        if best_moves:
            proposals[(x, y)] = random.choice(best_moves)
    return proposals

def resolve_and_update(state_grid, proposals):
    """
    Actualizar el estado del entorno basado en las propuestas de movimiento.
    Mueve el pájaro a las celdas propuestas, recolecta comida si es necesario y actualiza el historial de movimientos.

    """
    global trajectory, move_history, food_collected
    new_grid = state_grid.copy()
    move_targets = {}
    for from_cell, to_cell in proposals.items():
        if to_cell not in move_targets:
            move_targets[to_cell] = []
        move_targets[to_cell].append(from_cell)
    
    for to_cell, from_cells in move_targets.items():
        chosen_from = random.choice(from_cells)
        
        if state_grid[to_cell] == FOOD:
            food_collected += 1
        
        new_grid[to_cell] = BIRD
        new_grid[chosen_from] = EMPTY
        trajectory.append(to_cell)
        dx, dy = to_cell[0] - chosen_from[0], to_cell[1] - chosen_from[1]
        move_history.append((dx, dy))
        if len(move_history) > 10:
            move_history.pop(0)
    
    return new_grid

# ------------------------------
# Visualization
# ------------------------------
""""
Visualización del entorno y la trayectoria del pájaro.
Se utiliza matplotlib para crear una cuadrícula que representa el estado del entorno.
"""
fig, ax = plt.subplots(figsize=(9, 9))
cmap = plt.cm.get_cmap('Accent', 5)
im = ax.imshow(state_grid, cmap=cmap, vmin=0, vmax=4)

traj_x, traj_y = [], []
trajectory_line, = ax.plot([], [], 'k-', linewidth=2, label='Bird Trajectory')

legend_elements = [
    mpatches.Patch(color=cmap(0/4), label='Empty'),
    mpatches.Patch(color=cmap(1/4), label='Bird'),
    mpatches.Patch(color=cmap(2/4), label='Food'),
    mpatches.Patch(color=cmap(3/4), label='Prohibited Area'),
    mpatches.Patch(color=cmap(4/4), label='Destination'),
]
ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1, 1))

plt.title("Bird Flight Simulation with Compass Bias Implementation")

# ------------------------------
# Analysis
# ------------------------------
def compute_distance_traveled(traj):
    return sum(np.linalg.norm(np.array(traj[i]) - np.array(traj[i-1])) for i in range(1, len(traj)))

def save_results():
    """"
    Guarda los resultados de la simulación en archivos CSV y Excel.
    La trayectoria del pájaro se guarda en un archivo CSV y un resumen de la simulación se guarda en un archivo Excel.
    Incluye información sobre los pasos tomados, la distancia total recorrida, el promedio de longitud de paso, la comida recolectada,

    """
    df_traj = pd.DataFrame(trajectory, columns=["row", "col"])
    df_traj.to_csv("bird_trajectory.csv", index=False)

    steps_taken = len(trajectory) - 1
    distance_traveled = compute_distance_traveled(trajectory)
    unique_cells = len(set(trajectory))
    total_cells = grid_size[0] * grid_size[1]
    explored_percentage = (unique_cells / total_cells) * 100
    avg_step_length = distance_traveled / steps_taken if steps_taken > 0 else 0
    reached_destination = destination in trajectory

    summary_data = {
        "Steps Taken": [steps_taken],
        "Total Distance Traveled": [distance_traveled],
        "Average Step Length": [avg_step_length],
        "Food Collected": [food_collected],
        "Unique Cells Visited": [unique_cells],
        "Explored (%)": [explored_percentage],
        "Destination Reached": [reached_destination]
    }

    df_summary = pd.DataFrame(summary_data)
    df_summary.to_excel("bird_simulation_summary.xlsx", index=False)

    print("Trajectory saved as 'bird_trajectory.csv'")
    print("Summary saved as 'bird_simulation_summary.xlsx'")
    ipy_display(df_summary)

# ------------------------------
# Animation Update
# ------------------------------
ani = None

def update(frame):
    """"
    Actualiza el estado del entorno y la trayectoria del pájaro en cada fotograma de la animación.
    Si el pájaro llega al destino, detiene la animación y guarda los resultados.
    Si el fotograma es múltiplo de wind_change_interval, cambia la cuadrícula de viento.
    Proporciona nuevas propuestas de movimiento basadas en el estado actual del entorno y actualiza la cuadrícula de estado.
    Actualiza la imagen y la línea de trayectoria en la animación.

    """
    global state_grid, wind_grid, ani
    if trajectory[-1] == destination:
        ani.event_source.stop()
        on_animation_end()
        return [im, trajectory_line]

    if frame % wind_change_interval == 0 and frame > 0:
        wind_grid = np.random.randint(-5, 6, size=grid_size)

    oscillating = detect_oscillation(move_history)
    proposals = propose_moves(state_grid, wind_grid, oscillating)

    state_grid = resolve_and_update(state_grid, proposals)
    im.set_data(state_grid)

    traj_x = [pos[1] for pos in trajectory]
    traj_y = [pos[0] for pos in trajectory]
    trajectory_line.set_data(traj_x, traj_y)

    return [im, trajectory_line]

# ------------------------------
# Run Animation
# ------------------------------

def on_animation_end():
    print("Animation finished.")
    save_results()


ani = FuncAnimation(fig, update, frames=2000, interval=10, blit=True)
ani.save("bird_flight.gif", writer='pillow')



ani._stop = lambda *args, **kwargs: (FuncAnimation._stop(ani, *args, **kwargs), on_animation_end())

C:\Users\eloyl\AppData\Local\Temp\ipykernel_17196\3514493764.py:134: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('Accent', 5)


Animation finished.
Trajectory saved as 'bird_trajectory.csv'
Summary saved as 'bird_simulation_summary.xlsx'


Steps Taken  Total Distance Traveled  Average Step Length  Food Collected  \
0          588                    588.0                  1.0             390   

   Unique Cells Visited  Explored (%)  Destination Reached  
0                   528        0.8448                 True

Animation finished.
Trajectory saved as 'bird_trajectory.csv'
Summary saved as 'bird_simulation_summary.xlsx'


Steps Taken  Total Distance Traveled  Average Step Length  Food Collected  \
0          588                    588.0                  1.0             390   

   Unique Cells Visited  Explored (%)  Destination Reached  
0                   528        0.8448                 True

...

Steps Taken  Total Distance Traveled  Average Step Length  Food Collected  \
0          588                    588.0                  1.0             390   

   Unique Cells Visited  Explored (%)  Destination Reached  
0                   528        0.8448                 True

Animation finished.
Trajectory saved as 'bird_trajectory.csv'
Summary saved as 'bird_simulation_summary.xlsx'


Steps Taken  Total Distance Traveled  Average Step Length  Food Collected  \
0          588                    588.0                  1.0             390   

   Unique Cells Visited  Explored (%)  Destination Reached  
0                   528        0.8448                 True

Animation finished.
Trajectory saved as 'bird_trajectory.csv'
Summary saved as 'bird_simulation_summary.xlsx'


Steps Taken  Total Distance Traveled  Average Step Length  Food Collected  \
0          588                    588.0                  1.0             390   

   Unique Cells Visited  Explored (%)  Destination Reached  
0                   528        0.8448                 True